In [1]:
sc

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1588441152808_0001,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<SparkContext master=yarn appName=livy-session-0>

In [2]:
%%configure -f
{ "conf":{
"spark.pyspark.python": "python3",
"spark.pyspark.virtualenv.enabled": "true",
"spark.pyspark.virtualenv.type":"native",
"spark.pyspark.virtualenv.bin.path":"/usr/bin/virtualenv"
}}

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
1,application_1588441152808_0002,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
1,application_1588441152808_0002,pyspark,idle,Link,Link,✔


In [72]:
# Cargo los datos desde el S3
df=spark.read.csv('s3://marami26datasets/datasets/spark/Casos_positivos_de_COVID-19_en_Colombia.csv',inferSchema=True,header=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [73]:
############## Eliminacion de columnas #####################
# Obtengo todas las columnas
df.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['ID de caso', 'Fecha de notificación', 'Codigo DIVIPOLA', 'Ciudad de ubicación', 'Departamento o Distrito ', 'atención', 'Edad', 'Sexo', 'Tipo', 'Estado', 'País de procedencia', 'FIS', 'Fecha de muerte', 'Fecha diagnostico', 'Fecha recuperado', 'fecha reporte web']

In [74]:
# Actualizo el dataFrame eliminando las columnas ['ID de caso', 'Codigo DIVIPOLA', 'FIS', 'fecha reporte web']
df = df.select('Fecha de notificación', 'Ciudad de ubicación', 'Departamento o Distrito ', 'atención', 'Edad', 'Sexo', 'Tipo', 'Estado', 'País de procedencia', 'Fecha de muerte', 'Fecha diagnostico', 'Fecha recuperado')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [75]:
# Obtengo todas las columnas
df.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['Fecha de notificación', 'Ciudad de ubicación', 'Departamento o Distrito ', 'atención', 'Edad', 'Sexo', 'Tipo', 'Estado', 'País de procedencia', 'Fecha de muerte', 'Fecha diagnostico', 'Fecha recuperado']

In [101]:
############## Agregacion de columnas #####################
# Obtengo los tipos de datos de las columnas que me interesan
def get_dtype(df,colnames):
    for colname in colnames:
        print(colname+ " = "+[dtype for name, dtype in df.dtypes if name == colname][0])

get_dtype(df,['Fecha de notificación','Fecha recuperado','Fecha diagnostico','Fecha de muerte'])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Fecha de notificación = timestamp
Fecha recuperado = string
Fecha diagnostico = timestamp
Fecha de muerte = string

In [102]:
# Cambio el tipo de dato "string" a "timestamp" de las columnas ["Fecha recuperado", "Fecha de muerte"]
df = df.withColumn('Fecha de muerte', col('Fecha de muerte').cast('timestamp'))
df = df.withColumn('Fecha recuperado', col('Fecha recuperado').cast('timestamp'))
get_dtype(df,['Fecha de notificación','Fecha recuperado','Fecha diagnostico','Fecha de muerte'])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Fecha de notificación = timestamp
Fecha recuperado = timestamp
Fecha diagnostico = timestamp
Fecha de muerte = timestamp

In [127]:
# Calculo la diferencia de tiempo entre "Fecha diagnostico" y "Fecha de notificación"
# Agrego la nueva columna "Horas de espera" al dataFrame
# "Horas de espera" representa la horas que pasan desde que se notifican los sintomas, hasta que se obtiene el diagnostico
 
from pyspark.sql.functions import *

diff_secs_col = col("Fecha diagnostico").cast("long") - col("Fecha de notificación").cast("long")
df = df.withColumn( "Horas de espera", diff_secs_col/3600 )
df.select('atención', "Fecha de notificación", 'Fecha diagnostico',"Horas de espera").show(50,truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+---------------------+-------------------+---------------+
|atención  |Fecha de notificación|Fecha diagnostico  |Horas de espera|
+----------+---------------------+-------------------+---------------+
|Recuperado|2020-03-02 00:00:00  |2020-03-06 00:00:00|96.0           |
|Recuperado|2020-03-06 00:00:00  |2020-03-09 00:00:00|72.0           |
|Recuperado|2020-03-07 00:00:00  |2020-03-09 00:00:00|48.0           |
|Recuperado|2020-03-09 00:00:00  |2020-03-11 00:00:00|48.0           |
|Recuperado|2020-03-09 00:00:00  |2020-03-11 00:00:00|48.0           |
|Recuperado|2020-03-10 00:00:00  |2020-03-11 00:00:00|24.0           |
|Recuperado|2020-03-08 00:00:00  |2020-03-11 00:00:00|72.0           |
|Recuperado|2020-03-09 00:00:00  |2020-03-11 00:00:00|48.0           |
|Recuperado|2020-03-08 00:00:00  |2020-03-11 00:00:00|72.0           |
|Recuperado|2020-03-12 00:00:00  |2020-03-12 00:00:00|0.0            |
|Recuperado|2020-03-11 00:00:00  |2020-03-12 00:00:00|24.0           |
|Recup

In [126]:
# Calculo la diferencia de tiempo entre "Fecha recuperado" y "Fecha diagnostico"
# Agrego la nueva columna "Dias contagio" al dataFrame
# "Dias contagio" representa los dias que pasa un paciente contagiado hasta su recuperacion

diff_secs_col = col('Fecha recuperado').cast("long") - col("Fecha diagnostico").cast("long")
df = df.withColumn( "Dias contagiado", diff_secs_col/86400 )
df.select('atención', 'Fecha diagnostico', 'Fecha recuperado',"Dias contagiado").show(50,truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+-------------------+-------------------+---------------+
|atención  |Fecha diagnostico  |Fecha recuperado   |Dias contagiado|
+----------+-------------------+-------------------+---------------+
|Recuperado|2020-03-06 00:00:00|2020-03-13 00:00:00|7.0            |
|Recuperado|2020-03-09 00:00:00|2020-03-19 00:00:00|10.0           |
|Recuperado|2020-03-09 00:00:00|2020-03-15 00:00:00|6.0            |
|Recuperado|2020-03-11 00:00:00|2020-03-26 00:00:00|15.0           |
|Recuperado|2020-03-11 00:00:00|2020-03-23 00:00:00|12.0           |
|Recuperado|2020-03-11 00:00:00|2020-03-26 00:00:00|15.0           |
|Recuperado|2020-03-11 00:00:00|2020-03-17 00:00:00|6.0            |
|Recuperado|2020-03-11 00:00:00|2020-03-21 00:00:00|10.0           |
|Recuperado|2020-03-11 00:00:00|2020-03-23 00:00:00|12.0           |
|Recuperado|2020-03-12 00:00:00|2020-03-21 00:00:00|9.0            |
|Recuperado|2020-03-12 00:00:00|2020-03-31 00:00:00|19.0           |
|Recuperado|2020-03-12 00:00:00|20

In [131]:
############## Filtrados de datos #####################
# Filtro por paciente recuperado y
# dias de contagio diferentes de cero, ya que significa que notifico una vez se habia recuperado
df.filter((df['atención']=='Recuperado')&(df['Dias contagiado']!=0)).select('Edad', 'Sexo','Ciudad de ubicación','atención',"Dias contagiado").show(50,truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+----+-------------------+----------+---------------+
|Edad|Sexo|Ciudad de ubicación|atención  |Dias contagiado|
+----+----+-------------------+----------+---------------+
|19  |F   |Bogotá D.C.        |Recuperado|7.0            |
|34  |M   |Guadalajara de Buga|Recuperado|10.0           |
|50  |F   |Medellín           |Recuperado|6.0            |
|55  |M   |Medellín           |Recuperado|15.0           |
|25  |M   |Medellín           |Recuperado|12.0           |
|27  |F   |Itagüí             |Recuperado|15.0           |
|85  |F   |Cartagena de Indias|Recuperado|6.0            |
|22  |F   |Bogotá D.C.        |Recuperado|10.0           |
|28  |F   |Bogotá D.C.        |Recuperado|12.0           |
|36  |F   |Bogotá D.C.        |Recuperado|9.0            |
|42  |F   |Bogotá D.C.        |Recuperado|19.0           |
|74  |F   |Neiva              |Recuperado|28.0           |
|68  |F   |Neiva              |Recuperado|18.0           |
|48  |M   |Palmira            |Recuperado|8.0           

In [134]:
# Filtro por departamento y Fallecidos
# Para obtener los muertos en antioquia por causa del virus
#df.columns
df.filter((df['Departamento o Distrito ']=='Antioquia')&(df['atención']=="Fallecido")).select('Edad', 'Sexo','Ciudad de ubicación','atención', 'Fecha de muerte').show(50,truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+----+-------------------+---------+-------------------+
|Edad|Sexo|Ciudad de ubicación|atención |Fecha de muerte    |
+----+----+-------------------+---------+-------------------+
|80  |M   |San Francisco      |Fallecido|2020-04-24 00:00:00|
|91  |F   |Medellín           |Fallecido|2020-04-03 00:00:00|
|67  |M   |Medellín           |Fallecido|2020-04-18 00:00:00|
|74  |M   |Medellín           |Fallecido|2020-04-27 00:00:00|
|88  |F   |Bello              |Fallecido|2020-04-18 00:00:00|
+----+----+-------------------+---------+-------------------+

In [135]:
############## Agrupacion y consulta categorica #####################
# Numeros de casos por Departamento o distrito
df.groupBy('Departamento o Distrito ').count().show(50,False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------------------------------------------+-----+
|Departamento o Distrito                                |count|
+-------------------------------------------------------+-----+
|Buenaventura D.E.                                      |34   |
|Cundinamarca                                           |234  |
|Antioquia                                              |475  |
|Norte de Santander                                     |73   |
|Santander                                              |40   |
|Atlántico                                              |195  |
|Casanare                                               |18   |
|Huila                                                  |133  |
|Bolívar                                                |24   |
|Magdalena                                              |51   |
|Valle del Cauca                                        |979  |
|Cartagena D.T. y C.                                    |284  |
|Risaralda                              

In [145]:
# Numeros de fallecidos por edad
df.filter(df['atención']=="Fallecido").groupBy('Edad').count().show(50,False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+-----+
|Edad|count|
+----+-----+
|85  |4    |
|65  |11   |
|53  |1    |
|78  |4    |
|81  |6    |
|76  |10   |
|26  |1    |
|44  |4    |
|91  |4    |
|93  |1    |
|47  |1    |
|1   |2    |
|52  |1    |
|86  |3    |
|3   |1    |
|40  |2    |
|20  |1    |
|94  |2    |
|57  |7    |
|54  |3    |
|96  |1    |
|48  |4    |
|19  |1    |
|92  |1    |
|64  |9    |
|41  |1    |
|43  |1    |
|37  |2    |
|61  |5    |
|88  |4    |
|72  |9    |
|59  |5    |
|55  |3    |
|39  |2    |
|49  |4    |
|84  |2    |
|87  |3    |
|51  |2    |
|69  |6    |
|97  |2    |
|63  |6    |
|77  |4    |
|50  |5    |
|45  |1    |
|38  |1    |
|82  |5    |
|80  |4    |
|73  |12   |
|25  |1    |
|24  |1    |
+----+-----+
only showing top 50 rows

In [142]:
# Numeros de contagios por genero
df.groupBy('Sexo').count().show(50,False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+-----+
|Sexo|count|
+----+-----+
|F   |3249 |
|M   |3757 |
+----+-----+

In [147]:
############## Grabo los resultados en mi bucket de S3 #####################
# Direccion del bucket
write_uri='s3://marami26datasets/lab4/df_csv'
# Almacenamiento del dataFrame como un archivo csv
df.coalesce(1).write.format("csv").option("header","true").save(write_uri)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…